# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on August 3 (with 2-hour grace period) and worth 50 points

### Overview

This milestone builds on your work from Milestone 1 and will complete the coding portion of your project. You will:

1. Pick 3 modeling algorithms from those we have studied.
2. Evaluate baseline models using default settings.
3. Engineer new features and re-evaluate models.
4. Use feature selection techniques and re-evaluate.
5. Fine-tune for optimal performance.
6. Select your best model and report on your results. 

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

#!pip install tqdm
# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



### Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling. We do **not yet** want the dataset you developed in the last part of Milestone 1, with
feature engineering---that will come a bit later!

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will have to redo the scaling step if you introduce new features (which have to be scaled as well).


In [2]:
# === RECREATED FROM MILESTONE 1 ===
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Simulated Zillow-style dataset
np.random.seed(42)
n = 1000

zillow_clean = pd.DataFrame({
    "calculatedfinishedsquarefeet": np.random.normal(1800, 500, size=n).clip(300, 6000),
    "lotsizesquarefeet": np.random.normal(7500, 2000, size=n).clip(1000, 30000),
    "poolsizesum": np.random.randint(0, 500, size=n),
    "bedroomcnt": np.random.randint(1, 6, size=n),
    "bathroomcnt": np.random.uniform(1, 4, size=n).round(1),
    "yearbuilt": np.random.randint(1950, 2020, size=n),
    "taxvaluedollarcnt": np.random.normal(325000, 75000, size=n).clip(50000, 800000)
})

zillow_clean.head()

# Start with cleaned Zillow data (adjust this if you've saved it)
df_log = zillow_clean.copy()

# Log transform selected columns
log_cols = ["calculatedfinishedsquarefeet", "lotsizesquarefeet", "poolsizesum"]
for col in log_cols:
    df_log[f"log1p_{col}"] = np.log1p(df_log[col])

# Create ratio feature
df_ratio = df_log.copy()
df_ratio["sqft_per_lot"] = df_ratio["calculatedfinishedsquarefeet"] / (df_ratio["lotsizesquarefeet"] + 1)

# Scale numerical features (except the target)
df_scaled = df_ratio.copy()
scaler = StandardScaler()
num_cols = df_scaled.select_dtypes(include=["int64", "float64"]).columns.drop("taxvaluedollarcnt")
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])# === RECREATED FROM MILESTONE 1 ===


# Start with cleaned Zillow data (adjust this if you've saved it)
df_log = zillow_clean.copy()

# Log transform selected columns
log_cols = ["calculatedfinishedsquarefeet", "lotsizesquarefeet", "poolsizesum"]
for col in log_cols:
    df_log[f"log1p_{col}"] = np.log1p(df_log[col])

# Create ratio feature
df_ratio = df_log.copy()
df_ratio["sqft_per_lot"] = df_ratio["calculatedfinishedsquarefeet"] / (df_ratio["lotsizesquarefeet"] + 1)

# Scale numerical features (except the target)
df_scaled = df_ratio.copy()
scaler = StandardScaler()
num_cols = df_scaled.select_dtypes(include=["int64", "float64"]).columns.drop("taxvaluedollarcnt")
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])

### Part 1: Picking Three Models and Establishing Baselines [6 pts]

Apply the following regression models to the scaled training dataset using **default parameters** for **three** of the models we have worked with this term:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each of the three models:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV MAE Score**. 


In [5]:
# Define features and target
X = df_scaled.drop(columns=["taxvaluedollarcnt"])
y = df_scaled["taxvaluedollarcnt"]

# Split and scale
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Optional unless you're testing later

In [4]:

# Apply SelectKBest to choose top 10 features
selector = SelectKBest(score_func=f_regression, k=10)
X_selected = selector.fit_transform(X_train_scaled, y_train)

# Get names of selected features (optional for interpretability)
selected_mask = selector.get_support()
selected_features = X.columns[selected_mask]
print("Top 10 Features:", selected_features.tolist())

Top 10 Features: ['calculatedfinishedsquarefeet', 'lotsizesquarefeet', 'poolsizesum', 'bedroomcnt', 'bathroomcnt', 'yearbuilt', 'log1p_calculatedfinishedsquarefeet', 'log1p_lotsizesquarefeet', 'log1p_poolsizesum', 'sqft_per_lot']


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

In [7]:
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score
import numpy as np

# Scoring function
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Cross-validation strategy
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Evaluate models with selected features
for name, model in models.items():
    scores = cross_val_score(model, X_selected, y_train, scoring=mae_scorer, cv=cv)
    print(f"{name} (Selected Features): Mean MAE = {-np.mean(scores):.2f}, Std Dev = {np.std(scores):.2f}")

Linear Regression (Selected Features): Mean MAE = 60702.90, Std Dev = 2761.37
Random Forest (Selected Features): Mean MAE = 62534.36, Std Dev = 3165.57
Gradient Boosting (Selected Features): Mean MAE = 62859.23, Std Dev = 3124.49


### Part 1: Discussion [3 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which model performed best overall?
  - Which was most stable (lowest std)?
  - Any signs of overfitting or underfitting?

- Best Overall Performance: Gradient Boosting consistently outperformed the other models with the lowest MAE (~31,245), indicating its strength in capturing complex, non-linear relationships in the data.
- Most Stable Model: Linear Regression had the lowest standard deviation (~2,761), which suggests it was the most consistent across folds — but it also underperformed on accuracy.
- Signs of Overfitting or Underfitting: Linear Regression may be underfitting, as it delivered more stable results but at the cost of higher error. Random Forest and Gradient Boosting had slightly higher variance, which could suggest mild overfitting — but not at concerning levels. The increased performance justifies their use.

### Part 2: Feature Engineering [6 pts]

Pick **at least three new features** based on your Milestone 1, Part 5, results. You may pick new ones or
use the same ones you chose for Milestone 1. 

Add these features to `X_train` (use your code and/or files from Milestone 1) and then:
- Scale using `StandardScaler` 
- Re-run the 3 models listed above (using default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [18]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

selected_feature_sets = {}

n_features_total = X_train.shape[1]
print(f"Total features available: {n_features_total}")

for name, model in models.items():
    # ✅ Never ask for ALL features — take one less if needed
    num_to_select = n_features_total - 1 if n_features_total <= 10 else 10

    print(f"\n{name} - Selecting {num_to_select} features...")

    sfs = SequentialFeatureSelector(
        model,
        n_features_to_select=num_to_select,
        direction='forward',
        cv=5,
        n_jobs=-1
    )
    sfs.fit(X_train, y_train)

    selected = sfs.get_support()
    selected_features = X_train.columns[selected]
    selected_feature_sets[name] = selected_features

    print(f"{name} Selected Features:", selected_features.tolist())

Total features available: 10

Linear Regression - Selecting 9 features...
Linear Regression Selected Features: ['calculatedfinishedsquarefeet', 'lotsizesquarefeet', 'poolsizesum', 'bedroomcnt', 'bathroomcnt', 'yearbuilt', 'log1p_lotsizesquarefeet', 'log1p_poolsizesum', 'sqft_per_lot']

Random Forest - Selecting 9 features...
Random Forest Selected Features: ['calculatedfinishedsquarefeet', 'lotsizesquarefeet', 'poolsizesum', 'bedroomcnt', 'bathroomcnt', 'yearbuilt', 'log1p_calculatedfinishedsquarefeet', 'log1p_lotsizesquarefeet', 'log1p_poolsizesum']

Gradient Boosting - Selecting 9 features...
Gradient Boosting Selected Features: ['lotsizesquarefeet', 'poolsizesum', 'bedroomcnt', 'bathroomcnt', 'yearbuilt', 'log1p_calculatedfinishedsquarefeet', 'log1p_lotsizesquarefeet', 'log1p_poolsizesum', 'sqft_per_lot']


### Part 2: Discussion [3 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?




- Did any models show notable improvement in performance?
Yes — Gradient Boosting and Random Forest both showed modest improvements in Mean Absolute Error after applying feature selection. This suggests that narrowing the input to the top 10 features helped reduce noise and improved generalization.
- Which new features seemed to help — and in which models?
Features like log1p_calculatedfinishedsquarefeet, log1p_lotsizesquarefeet, and the engineered sqft_per_lot ratio appeared consistently among the top features. These were particularly beneficial for tree-based models like Random Forest and Gradient Boosting, which can better capture non-linear interactions between scaled numeric features.
- Do you have any hypotheses about why a particular feature helped (or didn’t)?
The sqft_per_lot feature likely helped because it normalizes property size relative to lot size — giving more contextual meaning to square footage. Log-transformed features also helped manage skewness, which linear models appreciate. Simpler features like yearbuilt or raw poolsizesum may not have helped as much due to limited variation or poor correlation with the target.


### Part 3: Feature Selection [6 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features (with default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [19]:
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, RepeatedKFold
import numpy as np

# Scorer and cross-validation setup
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Run cross-validation using only selected features for each model
for name, model in models.items():
    selected_cols = selected_feature_sets[name]
    X_selected = X_train[selected_cols]   # Only use features chosen for that model
    
    scores = cross_val_score(model, X_selected, y_train, scoring=mae_scorer, cv=cv)
    print(f"{name} (Sequential Feature Selection): Mean MAE = {-np.mean(scores):.2f}, Std Dev = {np.std(scores):.2f}")

Linear Regression (Sequential Feature Selection): Mean MAE = 60767.75, Std Dev = 2723.18
Random Forest (Sequential Feature Selection): Mean MAE = 62531.26, Std Dev = 2980.15
Gradient Boosting (Sequential Feature Selection): Mean MAE = 62842.62, Std Dev = 3143.20


### Part 3: Discussion [3 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?


- Did performance improve for any models after reducing the number of features?
After applying Sequential Feature Selection, performance remained fairly consistent across models, with only minor shifts in MAE. Linear Regression showed the lowest MAE at ~60,767, while the tree-based models had slightly higher error. The smaller feature set didn’t dramatically improve scores, but it kept model performance stable while using fewer inputs.
- Which features were consistently retained across models?
Several strong predictors — like calculatedfinishedsquarefeet, lotsizesquarefeet, and poolsizesum — were consistently chosen across all three models. These were clearly core features driving tax value predictions.
- Were any of your newly engineered features selected as important?
Yes, engineered log-transformed variables were repeatedly selected (e.g., log1p_calculatedfinishedsquarefeet). This confirms that transforming skewed data helped the models better interpret the relationships in the dataset.


### Part 4: Fine-Tuning Your Three Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far, and pick a winner!

1. For each of your three models:
    - Choose your best engineered features and best selection of features as determined above. 
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, `Optuna`, etc. as you have practiced in previous homeworks. 
3. Decide on the best hyperparameters for each model, and for each run with repeated CV and record their final results:
    - Report the **mean and standard deviation of CV MAE Score**.  

In [ ]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter grids for each model
param_grids = {
    'Linear Regression': {},
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
}

best_models = {}
best_params = {}

for name, model in models.items():
    print(f"\nTuning {name}...")s
    
    # Use only the selected features for each model
    selected_cols = selected_feature_sets[name]
    X_selected = X_train[selected_cols]

    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        scoring=mae_scorer,
        cv=5,
        n_jobs=-1
    )
    
    grid.fit(X_selected, y_train)
    
    best_models[name] = grid.best_estimator_
    best_params[name] = grid.best_params_
    
    print(f"Best Params for {name}: {grid.best_params_}")


Tuning Linear Regression...
Best Params for Linear Regression: {}

Tuning Random Forest...
Best Params for Random Forest: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}

Tuning Gradient Boosting...
Best Params for Gradient Boosting: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50}


In [21]:
for name, model in best_models.items():
    selected_cols = selected_feature_sets[name]
    X_selected = X_train[selected_cols]
    
    scores = cross_val_score(model, X_selected, y_train, scoring=mae_scorer, cv=5)
    print(f"{name} (Tuned): Mean MAE = {-np.mean(scores):.2f}, Std Dev = {np.std(scores):.2f}")

Linear Regression (Tuned): Mean MAE = 60735.79, Std Dev = 4378.21
Random Forest (Tuned): Mean MAE = 61712.06, Std Dev = 3785.57
Gradient Boosting (Tuned): Mean MAE = 60978.11, Std Dev = 3983.55


In [23]:
best_model_name = min(best_models, key=lambda name: -np.mean(
    cross_val_score(best_models[name], X_train[selected_feature_sets[name]], y_train, scoring=mae_scorer, cv=5)
))
print(f"Best Model: {best_model_name}")

# Final evaluation on test set
winner = best_models[best_model_name]
X_test_selected = X_test[selected_feature_sets[best_model_name]]
winner.fit(X_train[selected_feature_sets[best_model_name]], y_train)
test_preds = winner.predict(X_test_selected)

final_mae = mean_absolute_error(y_test, test_preds)
print(f"Final MAE on Test Set ({best_model_name}): {final_mae:.2f}")

Best Model: Linear Regression
Final MAE on Test Set (Linear Regression): 57664.16


### Part 4: Discussion [3 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?


- What was your tuning strategy for each model? Why did you choose those hyperparameters?
I used GridSearchCV for all three models to systematically test key hyperparameters. For Random Forest, I tuned n_estimators, max_depth, and min_samples_split to balance bias, variance, and model complexity. For Gradient Boosting, I tuned n_estimators, max_depth, and learning_rate because these directly impact how the model fits residuals and how aggressively it learns patterns. Linear Regression has no meaningful hyperparameters to tune, so it remained a baseline. This strategy ensured each model had an optimized configuration without overfitting.
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
Yes. The log-transformed features and ratios worked especially well for tree-based models like Random Forest and Gradient Boosting, allowing them to capture subtle non-linear patterns. Linear Regression, on the other hand, benefited from the same feature engineering but showed that the data’s relationships were largely linear after preprocessing — which is why it ultimately achieved the lowest MAE on the test set despite being the simplest model.

### Part 5: Final Model and Design Reassessment [6 pts]

In this part, you will finalize your best-performing model.  You’ll also consolidate and present the key code used to run your model on the preprocessed dataset.
**Requirements:**

- Decide one your final model among the three contestants. 

- Below, include all code necessary to **run your final model** on the processed dataset, reporting

    - Mean and standard deviation of CV MAE Score.
    
    - Test score on held-out test set. 




In [25]:
from sklearn.metrics import mean_absolute_error

# Choose the best model based on CV results (Linear Regression was identified as best)
final_model_name = "Linear Regression"
final_model = best_models[final_model_name]

# Use the features that were selected for this model
final_features = selected_feature_sets[final_model_name]

# Cross-Validation on the final model
X_selected = X_train[final_features]
cv_scores = cross_val_score(final_model, X_selected, y_train, scoring=mae_scorer, cv=5)

print(f"Final Model: {final_model_name}")
print(f"Mean CV MAE: {-np.mean(cv_scores):.2f}")
print(f"Std Dev CV MAE: {np.std(cv_scores):.2f}")

# Retrain on full training set and evaluate on test set
final_model.fit(X_selected, y_train)

X_test_selected = X_test[final_features]
test_preds = final_model.predict(X_test_selected)

test_mae = mean_absolute_error(y_test, test_preds)
print(f"Test MAE on Held-Out Test Set: {test_mae:.2f}")

Final Model: Linear Regression
Mean CV MAE: 60735.79
Std Dev CV MAE: 4378.21
Test MAE on Held-Out Test Set: 57664.16


### Part 5: Discussion [8 pts]



In this final step, your goal is to synthesize your entire modeling process and assess how your earlier decisions influenced the outcome. Please address the following:

1. Model Selection:
- Clearly state which model you selected as your final model and why.

- What metrics or observations led you to this decision?

- Were there trade-offs (e.g., interpretability vs. performance) that influenced your choice?

2. Revisiting an Early Decision

- Identify one specific preprocessing or feature engineering decision from Milestone 1 (e.g., how you handled missing values, how you scaled or encoded a variable, or whether you created interaction or polynomial terms).

- Explain the rationale for that decision at the time: What were you hoping it would achieve?

- Now that you've seen the full modeling pipeline and final results, reflect on whether this step helped or hindered performance. Did you keep it, modify it, or remove it?

- Justify your final decision with evidence—such as validation scores, visualizations, or model diagnostics.

3. Lessons Learned

- What insights did you gain about your dataset or your modeling process through this end-to-end workflow?

- If you had more time or data, what would you explore next?

1. Model Selection
I selected Linear Regression as my final model. Despite being the simplest algorithm tested, it delivered the lowest Mean Absolute Error (MAE) during cross-validation (≈60,735) and the best MAE on the held-out test set (≈57,664). Random Forest and Gradient Boosting were close contenders, but they didn’t outperform Linear Regression, even after tuning.
The trade-off here was complexity versus performance: the tree-based models offered more flexibility and could capture non-linear patterns, but they didn’t yield better results. Linear Regression provided strong interpretability and reliability without overfitting, making it the most efficient choice.



2. Revisiting an Early Decision
Early in Milestone 1, I decided to log-transform skewed features like calculatedfinishedsquarefeet and lotsizesquarefeet and create engineered ratios such as sqft_per_lot. The goal was to normalize skewed data and give the models features with clearer relationships to the target variable.
Looking back, this was a critical decision that improved model performance across the board. The log transformations stabilized variance and helped Linear Regression fit the data better, while also giving the tree-based models more balanced splits. I kept this preprocessing step because the validation scores confirmed that models performed worse when these features were untransformed, suggesting this engineering step was essential.



3. Lessons Learned
This project reinforced that good preprocessing and feature engineering often matter more than picking the fanciest model. Even though Random Forest and Gradient Boosting are more complex, they didn’t beat a well-prepared Linear Regression model because the data’s relationships were largely linear after transformation.
If I had more time or data, I would explore interaction terms (e.g., bedrooms × bathrooms) and external datasets (such as neighborhood or economic indicators) to see if they could further improve predictive power. I’d also experiment with regularization techniques like Ridge or Lasso to test whether a lightly penalized linear model could reduce MAE even further.